In [2]:
import tensorflow as tf

# Define the image input
image_input = tf.keras.Input(shape=(224, 224, 3), name="image_input")

# Preprocess the images as required by EfficientNetV2 models
preprocessed = tf.keras.applications.efficientnet_v2.preprocess_input(image_input)

# Load EfficientNetV2M as the base model using the preprocessed input tensor
base_model = tf.keras.applications.EfficientNetV2M(
    include_top=False,
    weights="imagenet",
    input_tensor=preprocessed
)

# Pass the preprocessed images through the base model
x = base_model.output

# Global average pooling for image features
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Define a second input for additional metadata
metadata_input = tf.keras.Input(shape=(33,), name="metadata_input")

# Process metadata features
metadata_features = tf.keras.layers.Dense(16, activation="relu")(metadata_input)

# Concatenate the image features with the metadata features
combined = tf.keras.layers.Concatenate()([x, metadata_features])

# Fully connected layers after concatenation
combined = tf.keras.layers.Dense(128, activation="relu")(combined)
combined = tf.keras.layers.Dropout(0.3)(combined)
combined = tf.keras.layers.Dense(64, activation="relu")(combined)

# Final output layer for binary classification (using sigmoid activation)
output = tf.keras.layers.Dense(1, activation="sigmoid")(combined)

# Define the full model with two inputs
model = tf.keras.Model(inputs=[image_input, metadata_input], outputs=output)

# Compile the model.
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name="accuracy"),
        tf.keras.metrics.AUC(name="AUC"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
    ]
)

# Display the model summary
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ image_input[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        648 │ rescaling_1[0][0] │
│                     │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │         96 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │      5,184 │ stem_activation[… │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 112, 112,  │         96 │ block1a_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 112, 112,  │          0 │ block1a_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_add (Add)   │ (None, 112, 112,  │          0 │ block1a_project_… │
│                     │ 24)               │            │ stem_activation[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_co… │ (None, 112, 112,  │      5,184 │ block1a_add[0][0] │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_bn  │ (None, 112, 112,  │         96 │ block1b_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_project_ac… │ (None, 112, 112,  │          0 │ block1b_project_… │
│ (Activation)        │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_drop        │ (None, 112, 112,  │          0 │ block1b_project_… │
│ (Dropout)           │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_add (Add)   │ (None, 112, 112,  │          0 │ block1b_drop[0][… │
│                     │ 24)               │            │ block1a_add[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1c_project_co… │ (None, 112, 112,  │      5,184 │ block1b_add[0][0] │
│ (Conv2D)            │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1c_project_bn  │ (None, 112, 112,  │         96 │ block1c_project_… │
│ (BatchNormalizatio… │ 24)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1c_project_ac… │ (None, 112, 112,  │          0 │ block1c_project_

 Total params: 53,325,269 (203.42 MB)

 Trainable params: 53,033,237 (202.31 MB)

 Non-trainable params: 292,032 (1.11 MB)

In [17]:
IMAGE_SIZE = (224, 224)

def _parse_function(proto):
    """Parse a single example from TFRecord."""
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.float32),
        "metadata": tf.io.FixedLenFeature([33], tf.float32),
    }

    example = tf.io.parse_single_example(proto, feature_description)

    # Decode image and preprocess
    image = tf.io.decode_jpeg(example["image"], channels=3)
    image = tf.image.resize(image, IMAGE_SIZE)

    # Normalize image (assuming already normalized, but ensuring consistency)
    image = tf.cast(image, tf.float32) / 255.0

    # Extract metadata and label
    metadata = example["metadata"]
    label = example["label"]

    return ({"image_input": image, "metadata_input": metadata}, label)

In [18]:
def load_tfrecord_dataset(tfrecord_paths, batch_size=32):
    """Load TFRecord dataset with given batch size."""
    dataset = tf.data.TFRecordDataset(tfrecord_paths)
    dataset = dataset.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [19]:
# Define paths to TFRecord files
train_tfrecords = [f'E:/Capstone Skin Cancer Project/Datasets/train.tfrecord']
val_tfrecords = [f'E:/Capstone Skin Cancer Project/Datasets/validation.tfrecord']

# Load datasets
train_dataset = load_tfrecord_dataset(train_tfrecords, batch_size=32)
val_dataset = load_tfrecord_dataset(val_tfrecords, batch_size=32)

# Callbacks for training optimization
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_AUC", patience=5, restore_best_weights=True, mode="max"
)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=3, verbose=1
)
class_weights = {0: 1.0, 1: 2.0}  # malignant is underrepresented

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=25,
    callbacks=[early_stopping, reduce_lr],
	class_weight=class_weights
)

Epoch 1/25
   1043/Unknown 9524s 9s/step - AUC: 0.9354 - accuracy: 0.0000e+00 - loss: 0.1073

KeyboardInterrupt: 

In [ ]:
test_tfrecords = [f'E:/Capstone Skin Cancer Project/Datasets/test.tfrecord']
test_dataset = load_tfrecord_dataset(test_tfrecords, batch_size=32)

# Evaluate the model
test_results = model.evaluate(test_dataset)
print("Test Results:", dict(zip(model.metrics_names, test_results)))

In [24]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [23]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print("Using AMD GPU!")
    except RuntimeError as e:
        print(e)
print(gpus)

[]
